In [2]:
!pip install thefuzz
!pip install streamlit
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
from thefuzz import process
import joblib



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
# Assuming the 'cast' column contains JSON-like strings, # downloading the CSV which contains the director and cast columns
df_5000_credits = pd.read_csv('/content/sample_data/tmdb_5000_credits.csv')
df_5000_credits.head(2)


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
# obtaining the CAST column from df_5000_credits

# Conver JSON cast to a list with the first 3 members of the cast.
df_5000_credits['cast'] = df_5000_credits['cast'].apply(ast.literal_eval)

# Create a new column with the first three cast members
df_5000_credits['cast_3'] = df_5000_credits['cast'].apply(lambda cast_list: [cast_member["name"] for cast_member in cast_list[:3]])
# Drop the original 'cast' column
df_5000_credits.drop(columns=['cast'], inplace=True)

# Now the 'cast' column contains only the first three cast members
print(df_5000_credits[['title', 'cast_3']])

                                         title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                                 cast_3  
0      [Sam Worthington, Zoe Saldana, Sigourney Weaver]  
1         [Johnny Depp, Orlando Bloom, Keira Knightley]  
2          [Daniel Craig, Christoph Waltz, Léa Seydoux]  
3          [Christian Bale, Michael Caine, Gary Oldman]  
4        [Taylor Kitsch, Lynn Collins, Samantha Morton]  
...                                                

In [ ]:
# obtaining the DIRECTOR column

# convert crew column from string to python list
df_5000_credits['crew'] = df_5000_credits['crew'].apply(ast.literal_eval)

# take from crew column, only the name of the director, as a list, even if there it's only one member
df_5000_credits['director'] = df_5000_credits['crew'].apply(lambda crew_list: [crew_member['name'] for crew_member in crew_list if crew_member["job"] == "Director"] )



In [ ]:
# DROP Original columns from credits dataframe
df_5000_credits.drop(columns=['crew','title'], inplace=True)


In [ ]:
# RENAME 'movie_id' column to 'id' to join later dataframes
df_5000_credits.rename(columns={'movie_id': 'id'}, inplace=True)
df_5000_credits.columns


Index(['id', 'cast_3', 'director'], dtype='object')

In [ ]:


#df = pd.read_csv('/content/sample_data/DMDB_subset_vote_count_higher_than_500_Vote_average_higher_than_7.1.csv')

In [ ]:
# We use chuck to save ram memory in the process of uploading the 1M movies DF (515 MB)

df1 = pd.read_csv('/content/sample_data/TMDB_movie_dataset_v11.csv')

chunksize = 100000
chunks = []

for chunk in pd.read_csv('/content/sample_data/TMDB_movie_dataset_v11.csv', chunksize = chunksize, iterator = True):
  chunks.append(chunk)

df1 = pd.concat(chunks)
del chunks

In [ ]:
# EDA of the first
df1.(describe)

,id,vote_average,vote_count,release_date,revenue,runtime,budget,popularity
count,1.174376e+06,1.174376e+06,1.174376e+06,971179.000000,1.174376e+06,1.174376e+06,1.174376e+06,1.174376e+06
mean,7.621705e+05,1.827318e+00,1.826509e+01,1999.940758,6.835395e+05,4.719594e+01,2.644234e+05,1.210490e+00
std,4.069846e+05,2.997631e+00,3.133688e+02,26.625620,1.845487e+07,6.157044e+01,5.100897e+06,7.465964e+00
min,2.000000e+00,0.000000e+00,0.000000e+00,1800.000000,-1.200000e+01,-2.800000e+01,0.000000e+00,0.000000e+00
25%,4.221728e+05,0.000000e+00,0.000000e+00,1989.000000,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e-01
50%,7.684505e+05,0.000000e+00,0.000000e+00,2011.000000,0.000000e+00,2.100000e+01,0.000000e+00,6.000000e-01
75%,1.126114e+06,4.500000e+00,1.000000e+00,2019.000000,0.000000e+00,8.800000e+01,0.000000e+00,8.740000e-01
max,1.430544e+06,1.000000e+01,3.449500e+04,2099.000000,5.000000e+09,1.440000e+04,1.000000e+09,2.994357e+03


In [ ]:
# MERGING BOTH DFs on 'id'
df_merge = pd.merge(df1, df_5000_credits, on='id', how='inner')

#filtering vote average > 6.5 and more than 50 votes, final df has 2000 movies.

df2 = df_merge[(df_merge['vote_average'] > 6.5 ) & (df_merge['vote_count'] > 50) ]
df2.shape


(2000, 26)

In [ ]:
df2.isnull().sum()

,0
id,0
title,0
vote_average,0
vote_count,0
status,0
release_date,0
revenue,0
runtime,0
adult,0
backdrop_path,2


In [ ]:
# coppying and DROPPING unnecessary columns
df = df2.copy()
df = df.drop(['status','backdrop_path','homepage','imdb_id','production_companies'], axis=1)

In [ ]:
df.head(1)

,id,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,False,160000000,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."


In [ ]:
# Defininng cols categories for data processing
cols_list = ['genres', 'keywords', 'spoken_languages', 'production_countries','director','cast_3']
cols_value = ['vote_average','vote_count','release_date','revenue','runtine','budget','popularity']
cols_text = ['overview','tagline']

In [ ]:
# strings to list for data processing

def cols_str_transformation_to_python_list(df):
  cols_list = ['genres', 'keywords', 'spoken_languages', 'production_countries']


  for col in cols_list:
    if col in df.columns: # I ensure de column exist
      #df_filtered[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
      df[col] = df[col].apply(lambda x: x.split(', ') if isinstance(x,str) else x)


  return df  # Debe ser un valor que Pandas pueda almacenar en una Serie



df_1 = cols_str_transformation_to_python_list(df)


In [ ]:
df_2 = df_1.copy()

In [ ]:
# taking just the year from the "release_date" column and converting it to int for later manipulation
for col in df_2['release_date']:
  df_2['release_date'] = df['release_date'].apply(lambda x: int(x.split('-')[0]) if isinstance(x,str) else x)

print(df_2['release_date'][0])
print(type(df_2['release_date'][1500]))

2010
<class 'numpy.int64'>


In [ ]:
# DROPPING DUPLICATES in cols_list

df_3 = df_2.copy()
cols_list = ['genres', 'keywords', 'spoken_languages', 'production_countries']
for col in cols_list:
  if col in df_3.columns:
    df_3[col].drop_duplicates()




      id      title  vote_average  vote_count  release_date    revenue  \
0  27205  Inception         8.364       34495          2010  825532764   

   runtime  adult     budget original_language  ...  \
0      148  False  160000000                en  ...   

                                            overview popularity  \
0  Cobb, a skilled thief who commits corporate es...     83.952   

                        poster_path                               tagline  \
0  /oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg  Your mind is the scene of the crime.   

                                 genres  \
0  [Action, Science Fiction, Adventure]   

                         production_countries  \
0  [United Kingdom, United States of America]   

                       spoken_languages  \
0  [English, French, Japanese, Swahili]   

                                            keywords  \
0  [rescue, mission, dream, airplane, paris, fran...   

                                              cast_3            

In [ ]:
# Handling N/A, as far as there are not many, and mostly in no super relevant columns, we give them some kind of string format/info


cols_with_na = ['overview','poster_path','tagline','genres','production_countries','spoken_languages','keywords']
for col in cols_with_na:
  if col in df_3.columns:
    df_3[col] = df_3[col].fillna('No '+col)

#For those movies without release date, we give them the year "2000"
for col in df_3['release_date']:
  df_3['release_date'] = df_3['release_date'].fillna(str('2000'))

df_3.isnull().sum()

,0
id,0
title,0
vote_average,0
vote_count,0
release_date,0
revenue,0
runtime,0
adult,0
budget,0
original_language,0


In [ ]:
# Here I download the df_3 dataframe, so as we are done with the EDA and the data processing, and
df_3.to_csv("df_4.csv", index=False)  # Saves without the index

In [ ]:
# So we upload the the polished DF, like that we save  RAM memory, then as well here we start the analysis.
df_4 = pd.read_csv('/content/sample_data/df_4.csv')
df_4.head(1)



,id,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,original_language,...,overview,popularity,poster_path,tagline,genres,production_countries,spoken_languages,keywords,cast_3,director
0,27205,Inception,8.364,34495,2010,825532764,148,False,160000000,en,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"['Action', 'Science Fiction', 'Adventure']","['United Kingdom', 'United States of America']","['English', 'French', 'Japanese', 'Swahili']","['rescue', 'mission', 'dream', 'airplane', 'pa...","['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ...",['Christopher Nolan']


In [ ]:

# I import the class MultiLabelBinarizer
#from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_3_mlb = df_4.copy()

cols_list = ['title','genres','keywords', 'spoken_languages', 'production_countries','director','cast_3','title']

# Initialize an empty DataFrame to hold the encoded data
#df_mlb = pd.DataFrame(index=df_3_mlb.index) #use the index from the df_3_mlb


for col in cols_list:
  encoded_binary_narray = mlb.fit_transform(df_3_mlb[col])
  categories = mlb.classes_
  #print(categories)
  #So each categorie is gonna be printed this way
  encoded_df = pd.DataFrame(encoded_binary_narray, columns=[f"{col}_{cat}" for cat in categories],index=df_3_mlb.index)
  df_3_mlb = pd.concat([df_3_mlb, encoded_df], axis=1) #concatenate horizontally

df_mlb = df_3_mlb.drop(columns = ['vote_average', 'vote_count', 'release_date', 'revenue', 'runtime', 'adult', 'budget','overview', 'popularity', 'poster_path', 'tagline','genres', 'production_countries', 'spoken_languages', 'keywords','production_countries'])

print(df_mlb.shape)
#print(df_mlb.head(1))




(2000, 553)


In [ ]:
# preprocessing text from columns 'overview' 'tagline' columns


import re

def preprocess(text):
    text = re.sub(r'\W', ' ', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.lower()  # Convert to lowercase
    return text

overview_col_text_preprocessing = df_4['overview'].apply(preprocess)
tagline_col_text_preprocessing = df_4['tagline'].apply(preprocess)

print(overview_col_text_preprocessing[0])
print(tagline_col_text_preprocessing[0])


cobb a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible inception the implantation of another person s idea into a target s subconscious 
your mind is the scene of the crime 


In [ ]:
# TF-IDF, create a df from both columns strings, tagline and overview.

#from sklearn.feature_extraction.text import TfidfVectorizer
cols_text = ['tagline','overview']
vectorizer = TfidfVectorizer(max_features=10000) # maybe on the final model take out "max_features=5000"
df_3_vectorizer = df_4.copy()
df_td_idf = pd.DataFrame(index = df_3_vectorizer.index) # empty DF

#df_vectorizer_td_idf = pd.DataFrame

# I create an exparse matrix
for col in cols_text:
    vectorizer_td_idf = vectorizer.fit_transform(df_3_vectorizer[col])
    # to join together the four Df, I need to turn the sparse matrix to a dense matrix with toarray()
    tfidf_dense = vectorizer_td_idf.toarray()

    tfidf_classes = vectorizer.get_feature_names_out()

    df_tfidf_dense = pd.DataFrame(tfidf_dense,columns = [f"{col}_{clas}" for clas in tfidf_classes],index = df_3_vectorizer.index)
    df_td_idf = pd.concat([df_td_idf,df_tfidf_dense],axis=1)

df_td_idf.shape

(2000, 12704)

In [ ]:
cols_value = ['vote_average','vote_count','release_date','revenue','runtine','budget','popularity']



In [ ]:
#if 'release_date' in df_3.columns is type(int):
for number in df_4['release_date']:
  if type(number) == type(np.float64):
    df_3['release_date'] = df_3['release_date'].astype(int)

print(type(df_4['release_date'][0]))

<class 'numpy.int64'>


In [ ]:
df_3_scaled = df_4.copy()

In [ ]:
df_scaled = df_4[['vote_average','vote_count','release_date','revenue','runtime','budget','popularity']].copy()

In [ ]:
# Instead of creating a new dataframe then concatenating the transformed values, we are gonna add these new scaled values to the original DF then, drop the non scaled values

#from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Apply Log Transformation + StandardScaler for skewed values
for col in ["vote_count","revenue","budget","popularity"]:
    df_scaled[f"{col}_scaled"] = np.log1p(df_scaled[col])

df_scaled[["vote_count_scaled","revenue_scaled","budget_scaled","popularity_scaled"]] = scaler.fit_transform(df_scaled[["vote_count_scaled","revenue_scaled","budget_scaled","popularity_scaled"]])
df_scaled.columns


Index(['vote_average', 'vote_count', 'release_date', 'revenue', 'runtime',
       'budget', 'popularity', 'vote_count_scaled', 'revenue_scaled',
       'budget_scaled', 'popularity_scaled'],
      dtype='object')

In [ ]:
# **3. Apply RobustScaler for runtime (outliers)**
#from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
df_scaled[['runtime_scaled']] = robust_scaler.fit_transform(df_scaled[['runtime']])
df_scaled.shape

(2000, 12)

In [ ]:
# Apply MinMaxScaler to small range values
#from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
df_scaled[['vote_average_scaled','release_date_scaled']] = minmax_scaler.fit_transform(df_scaled[['vote_average', 'release_date']])
df_scaled.columns


Index(['vote_average', 'vote_count', 'release_date', 'revenue', 'runtime',
       'budget', 'popularity', 'vote_count_scaled', 'revenue_scaled',
       'budget_scaled', 'popularity_scaled', 'runtime_scaled',
       'vote_average_scaled', 'release_date_scaled'],
      dtype='object')

In [ ]:
df_scaled.drop(columns=['vote_average','vote_count','revenue','runtime','budget','popularity'],inplace=True)


In [ ]:

combined_df_3 = pd.concat([df_mlb, df_td_idf, df_scaled],axis=1)
combined_df_3 = combined_df_3.fillna(0)
combined_df_3["release_date_scaled"]
combined_df_3.shape
combined_df_3.columns


Index(['id', 'title', 'original_language', 'original_title', 'cast_3',
       'director', 'title_ ', 'title_!', 'title_&', 'title_'',
       ...
       'overview_zyklon', 'overview_émigré', 'release_date',
       'vote_count_scaled', 'revenue_scaled', 'budget_scaled',
       'popularity_scaled', 'runtime_scaled', 'vote_average_scaled',
       'release_date_scaled'],
      dtype='object', length=13265)

In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity

# Assuming you have your DataFrames with the encoded features
# df_genres, df_keywords, df_overview, df_countries, df_languages, df_numerical

# Calculate cosine similarity matrices cols_list = ['genres','keywords', 'spoken_languages', 'production_countries']

similarity_genres = cosine_similarity(combined_df_3.filter(like="genres",axis=1))
similarity_keywords = cosine_similarity(combined_df_3.filter(like="keywords",axis=1))
similarity_spoken_languages = cosine_similarity(combined_df_3.filter(like="spoken_languages",axis=1))
similarity_production_countries = cosine_similarity(combined_df_3.filter(like="production_countries",axis=1))
similarity_title = cosine_similarity(combined_df_3.filter(like="title_",axis=1))

#
similarity_overview = cosine_similarity(combined_df_3.filter(like="overview",axis=1))
similarity_tagline = cosine_similarity(combined_df_3.filter(like="tagline",axis=1))
#

similarity_vote_average = cosine_similarity(combined_df_3.filter(like="vote_average",axis=1))
similarity_vote_count = cosine_similarity(combined_df_3.filter(like="vote_count",axis=1))
similarity_release_date = cosine_similarity(combined_df_3.filter(like="release_date",axis=1))
similarity_revenue = cosine_similarity(combined_df_3.filter(like="revenue",axis=1))
similarity_runtime = cosine_similarity(combined_df_3.filter(like="runtime",axis=1))
similarity_budget = cosine_similarity(combined_df_3.filter(like="budget",axis=1))
similarity_popularity = cosine_similarity(combined_df_3.filter(like="popularity",axis=1))

similarity_director = cosine_similarity(combined_df_3.filter(like="director_",axis=1))
similarity_cast_3 = cosine_similarity(combined_df_3.filter(like="cast_3_",axis=1))



similarity_cast_director_title =  cosine_similarity(combined_df_3.filter(like="director_",axis=1).join(combined_df_3.filter(like="cast_3_",axis=1)).join(combined_df_3.filter(like="title_",axis=1)))




In [ ]:
def similarity_combined():
  # Define weights for movie search
  weight_title = 0.3
  weight_genres = 0.05
  weight_keywords = 0.05
  weight_production_countries = 0.05
  weight_spoken_languages = 0.05
  weight_director = 0.2
  weight_cast_3 = 0.15


  weight_overview = 0.05
  weight_tagline = 0.05

  weight_vote_average = 0.05
  weight_vote_count = 0.05
  weight_release_date = 0.00
  weight_revenue = 0.00
  weight_runtime = 0.00
  weight_budget = 0.00
  weight_popularity = 0.00

  # Combine similarity matrices using weighted average
  similarity_combined = (
      (weight_title * similarity_title) +
      (weight_genres * similarity_genres) +
      (weight_keywords * similarity_keywords) +
      (weight_production_countries * similarity_production_countries) +
      (weight_spoken_languages * similarity_spoken_languages) +
      (weight_overview * similarity_overview) +
      (weight_tagline * similarity_tagline) +
      (weight_vote_average * similarity_vote_average) +
      (weight_vote_count * similarity_vote_count) +
      (weight_release_date * similarity_release_date) +
      (weight_revenue * similarity_revenue) +
      (weight_runtime * similarity_runtime) +
      (weight_budget * similarity_budget) +
      (weight_popularity * similarity_popularity)+
      (weight_director * similarity_director)+
      (weight_cast_3 * similarity_cast_3)
  )

  return similarity_combined

similarity_combined = similarity_combined()

'''
file_path = 'similarity_matrix.joblib'
joblib.dump(similarity_combined, file_path)
'''


"\nfile_path = 'similarity_matrix.joblib'\njoblib.dump(similarity_combined, file_path)\n"

In [ ]:
similarity_movies = similarity_combined[222]
print(similarity_cast_director_title[0])

[1.         0.7073172  0.70450233 ... 0.62303454 0.51241419 0.46106249]


In [ ]:


def get_recommendations_by_movie_title(movie_title, df_4, similarity_combined, n= 10):



  # check if movie is there, otherwise get the closest match, then get index
  choices = df_4['title']

  if movie_title in df_4['title'].values:
        movie_index = df_4[df_4['title'] == movie_title].index[0]
        #return movie_index


    # Fuzzy match on the combined column
  best_match = process.extractOne(movie_title, df_4['title'].values, score_cutoff=60)
  #print(df_4['combined_search'][0])
  if best_match:
      movie_index = df_4[df_4['title'] == best_match[0]].index[0]
     # return movie_index



  # Get similarity scores for this movie, this line is chossing a row of the big similarity_combined matrix, every score is the comparison bettwen two movies, and we get an array of them
  similarity_scores = similarity_combined[movie_index]

  # Get indices of top_n most similar movies (excluding itself),np.argsort() orders the indexes in a given order, [::-1] means we want them in descending order
  # [1:top_n + 1] means we want the second index (the first is the movie itself) to the n+1
  similar_movies_indexes = np.argsort(similarity_scores)[::-1][1:n + 1]

  ids = []
  for i in similar_movies_indexes:
    ids.append(int(df_4.iloc[i]['id']))

  return ids


recommendations_movie = get_recommendations_by_movie_title("Star wars", df_4, similarity_combined, n= 10)
print(recommendations_movie)



[1895, 1894, 1891, 10673, 2270, 203, 563, 862, 106646, 13475]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Initialize TF-IDF vectorizer but do NOT fit here.


def get_keywords_recommendations(keywords, n, df_4):


    # Process the input keywords
    keywords = " ".join(keywords.split())

    tfidf = TfidfVectorizer(stop_words='english')
    # Combine relevant movie data for TF-IDF fitting
    movie_data = df_4['director'].astype(str) + df_4['cast_3'].astype(str) + df_4['title'].astype(str)

    # Fit and transform TF-IDF on movie data and keywords together
    all_text = [keywords] + movie_data.tolist()  # Combine keywords and movie data

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(all_text)

    # Split the matrix to get keyword vector and movie vectors
    key_tfidf = tfidf_matrix[0]  # Vector for keywords (first row)
    movie_tfidf = tfidf_matrix[1:] # Vectors for movies (remaining rows)

    # Compute cosine similarity
    result = cosine_similarity(key_tfidf, movie_tfidf)

    # Get top n most similar movies (excluding itself)
    similar_key_movies = sorted(enumerate(result[0]), key=lambda x: x[1], reverse=True)

    # Extract top recommendations names
    recomm = [df_4.iloc[i[0]].title for i in similar_key_movies[:n]] # Index starts at 0

    # Extract top recommendations ids
    recomm_ids = [int(df_4.iloc[i[0]].id) for i in similar_key_movies[:n]] # Index starts at 0
    return recomm_ids

# Example Usage:
recommendations_keywords = get_keywords_recommendations("star wars", 10, df_4)
print(recommendations_keywords)

[11, 1894, 1895, 26748, 12508, 13475, 54138, 188927, 29963, 154]


In [ ]:

def recommend_movies_by_genre(df, genre, sorting, top_n=10):



  # Filter the DataFrame


  df['genres'] = df['genres'].apply(ast.literal_eval) # turning strings into a list
  # getting the first appearence of each list to make the search with apply(), then making the filtering
  filtered_df = df[df['genres'].apply(lambda x: x[0] == genre if isinstance(x, list) and len(x) > 0 else False)]


  # Randomly sample with padas.sample() top_n is number of rows to be selected, len(filtered_df) is the max of rows to be selected
  recommendations = filtered_df.sample(n=min(top_n, len(filtered_df)))
  #sorting by the parameter introduced
  recommendations = recommendations.sort_values(by=[sorting], ascending=False)

  return recommendations['id'].tolist()

# Example usage with 'df_4' as the dataset:
recommendations_random = recommend_movies_by_genre(df_4, 'Comedy', "release_date", top_n=10)
print(recommendations_random)


[51540, 39781, 10947, 926, 22796, 13685, 2925, 4032, 771, 957]
